<a href="https://colab.research.google.com/github/VisionLogic-AI/Unsupervised_learning/blob/master/Unsupervised_Learning(Clustering_Algorithms).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Clustering Algorithms 
Before we can perform clustering, we must perform dimensionality reduction using the most powerful unupervised learning algorithms at our disposal.

In this notebook, we are going to use the MNIST dataset as an example dataset to work with.

In [ ]:
#PCA
from sklearn.decomposition import PCA

n_components= 784
whiten= False
random_state= 2018

pca= PCA(n_components= n_components, whiten= whiten, random_state= random_state)

x_train_pca= pca.fit_transform(x_train)   #training set from our previous notebook 
x_train_pca= pd.DataFrame(data= x_train_pca, index= x_train.index)

Although we did not reduce the dimensiionality, we will designate the number of principal components we will use during the clustering stage, effectively reducing the dimensionality.<br>
In other worda, we will apply our chosen principal components during the clustering stage of the ML pipleine.

Now leys move on to clustering. Clustering has three main algorithms:<br>
- k means 
- hierarchical clustering
- DBSCAN

#K-Means Clustering
The objectve of clustering is to id distinct group in a dataset such that the observations within a group are similar to each other but different from observations in other groups.<br>

***In k-means clustering, we specifiy the number of desired clusters k, and the algorithm will assign each observation to exaclty one of these k-clusters.***<br>
*The algorithm optimizes the groups by minimizing the "within the cluster variation"...also known as intertia, such that the su of the within the cluster variations across all k clusters is as small as possible.

Clustering Steps:<br>
1. We need to set the umber of clusters we would like **(n_clusters)**
2. the number of initializations the algorithm will run to reassign observations in order to minimize inertia**(max_iter)**
3. the tolerance to declare convergence**(tol)**

In this example, we will keep the default values for the number of initializations to (10), maximum number of iterations (300), and tolerance (0.0001)<br>
*Also, for now we will use the first 100 principal components from PCA as the *cutoff*.

To test how the number of clusters we designate affects the inertia measure, lets run k means for cluster sizes 2 through 20 and record the inertia for each.


In [ ]:
#k means - Inertia as the number of clusters varies

from sklearn.cluster import KMeans

n_clusters = 10
n_init= 10
max_iter= 300
tol= 0.0001
random_state = 2018
n_jobs= 2

kmeans_inertia= pd.DataFrame(data= [], index= range(2,21), columns= ['intertia'])   #range between 2-20

for n_clusters in range(2,21):
  kmeans= KMeans(n_clusters= n_clusters, n_init= n_init, max_iter= max_iter,
                 tol= tol, random_state= random_state, n_jobs= n_jobs)
  cutoff= 99
  kmeans.fit(x_train_pca.loc[:,0: cutoff])
  kmeans_inertia.loc[n_clusters]= kmeans.inertia_
  

The figure shows the inertia decrease as the number of clusters increases...this makes sense.<br>
The more clusters we have, the greater the homogeneity among observations within each cluster.<br>
However, fewer clusters are easier to work with than more, so finding the right number of clusters to generate is an important consideration when running k-means.

#Evaluating the Clustering Results
To demonstrate how k-means works and how increasing the number of clusters results in more homogenpous clusters, lets define a function that can analyze the results.<br>
*The cluster assignments generatedby the clustering algorithm will be stored in a pandas dataframe called clusterDF.

Lets count the number of observations within each cluster and store these in  a Pandas dataframe called countybycluster



In [ ]:
def analyze_cluster(clusterDF, labelsDF):
  countbycluster= pd.DataFrame(data= clusterDF['cluster'].value_counts())
  countbycluster.reset_index(inplace= True, drop= False)
  countbycluster.columns= ['cluster', 'cluster_count']

Next lets join the clusterDF with the true labels array, which we will call labelsDF

In [ ]:
preds= pd.concat([labelsDF, clusterDF], axis= 1)
preds.columns= ['truelabel', 'cluster']

#lets also count the number of observations for each true label in the training set 
#this won't change but is good for us to know

countbylabel= pd.DataFrame(data= preds.groupby('truelabel').count())

For example, if a given cluster has three thousand observations:
  - two thousand may represent the number 2
  - five hundred may represent the number 1
  - three nundred may represent the number 0
  - the remaining two hundred may represent the number 9

Once we calculate these, we will store the count for the most frequently occuring number for each cluster.

Lets use this example above

In [ ]:
countmostfreq= pd.DataFrame(data = preds.groupby('cluster').agg(lambdax:x.value_counts().iloc[0]))
countmostfreq.reset_index(inplace= True, drop= False)
countmostfreq.columns= ['cluster', 'CountMostFrequent']

Finally, we will judge the success of each clustering run based on how tightly grouped the observations are within each cluster.<br>
For example, the example above has two thousand observations that have the same label out of a total of three thousand observations in the cluster.

This cluster is not great since we ideally want to group similar observations together in the same cluster and exclude dissimilar ones.

Lets define the overall accuracy of the clustering as the sum of the counts of the most frequently occuring observations across all the clusters divided by the total number of observations in the training set. (ex. 50,000)

In [ ]:
#code
accuracyDF= countmostfreq.merge(countbycluster, left_on= 'cluster', right_on= 'cluster')
overall_accuracy= accuracyDF.countmostfreq.sum()/ accuracyDF.clusterCount.sum()
#we cal also access the accuracy by cluster
accuracy_by_label= accuracyDF.countmostfreq/ accuracyDF.clusterCount

#K-Means Accuracy
Lets now perform these expirements we did earlier, but instead of calculating the inertia, we will instead focus on the overall homogeneity of the digits dataset

In [ ]:
#k-means Accuracy as the number of clusters varies

n_clusters= 5
n_init= 10
max_iter= 300
tol= 0.0001
random_state= 2018
n_jobs= 2

kmeans_inertia= pd.DataFrame(data= [], index= range(2,21), columns= ['inertia'])
overall_accuracy_kmeansDF= pd.DataFrame(data= [], index= range(2,21), columns= ['overall_accuracy'])

for n_clusters in range(2,21):
  kmeans= KMeans(n_clusters= n_clusters, n_init= n_init, max_iter= max_iter, 
                 tol= tol, random_state= random_state, n_jobs= n_jobs)
  cutoff = 99
  kmeans.fit(x_train_pca.loc[:, 0: cutoof])
  kmeans_inertia.loc[n_clusters]= kmeans_inertia_

  x_train_kmeans_clustered= kmeans.predict(x_train_pca.loc[:, 0: cutoof])
  x_train_kmeans_clustered= pd.DataFrame(data= x_train_kmeans_clustered, index= x_train.index, columns= ['cluster'])
  
  countybycluster_kmeans, countybylabel_kmeans, countmostfreq_kmeans, accuracydf_kmeans, overallaccuracy_kmeans, accuracybylabels_kmeans= analyze_cluster(
      x_train_kmeans_clustered, y_train)
  
  overall_accuracy_kmeansDF.loc[n_clusters]= overallaccuracy_kmeans

As we can see, the accuracy improves as the number of clusters increases.<br>
In other words, clusters become more homogeneous as we increase the number of clusters because each cluster becomes smaller and more tightly formed.

Accuracy by cluster may vary quite a bit, with some clusters exibiting a high degree of homogeneity and others exibiting less.

For example, in some clusters, over 90% of the images have the same digit, in other clusters, less than 50% of the images have the same digit.

#K-Means and Number of Principal Components
Let prform another experiment but this time we will assess how varying the number of principal components we use in the clustering algorithm impacts the homogeneity of the clusters (defind as accuracy)

In the experimnets earlier, we used one hundred principal components, derived from the normal PCA.<br>
(Recall that the original number of dimensions within the dataset is 784).

Now, if PCA does a good job of capturing the salient information or structure within the data, as compactly as possible, the clustering alogorithm will have an easy time grouping similar images together, regardless of whether we use 10-50 components or or hundreds of them.

Lets pass along 10, 50, 100, 200, 300, 400, 500, 600, and 700 components out of the 784 total observations and then gauge the accuracy of each clustering experiment.

In [ ]:
#k-means (Accuracy as the number of components varies)
n_cluster 20
n_init= 10
max_iter= 300
tol= 0.0001
random_state= 2018
n_jobs= 2

kmeans_inertia= pd.DataFrame(data= [], index= [9, 49, 99, 199, 299, 399, 499,
                                               599, 699, 784], columns= ['inertia'])
overallaccuracy_kmeansDF= pd.DataFrame(data= [], index[9, 49, 99, 199, 299, 399, 
                                                       499, 599, 699, 784], columns= ['overall_accuracy'])

for cutoffnumber in [9, 49, 99, 199, 299, 399, 499, 599, 699, 784]:
  kmeans= KMeans(n_clusters= n_clusters, n_init= n_init, max_iter= max_iter,
                 tol= tol, random_state= random_state, n_jobs= n_jobs)
  cutoff= cutoffnumber
  kmeans.fit(x_train_pca.loc[:, 0:cutoff])
  kmeans_inertia.loc[cutoff]= kmeans.inertia_

x_train_kmeansclustered= kmeans.predict(x_train_pca.loc[:, 0: cutoff])
x_train_kmeansclustered= pd.DataFrame(data= x_train_kmeansclustered, index= x_train.index, columns= ['cluster'])

countybycluster_kmeans, countbylabels_kmeans, countmostfreq, accuracydf_kmeans, overallaccuracy_kmeans, accuracybylabels_kmeans= analyzecluster(
    x_train_kmeansclustered, y_train)

overallaccuracy_kmeansDF.loc[cutoff]= overallaccuracy_kmeans

As we can see this plot supports out hypothesis.<br>
As the number of principal components varies from 10-784, the clustering accuracy remains stable and consistent around 70%.<br>

This is one reason why clustering should be performed on dimensionality reduced datasets- the clustering algorithms generally perform better, both in terms of time and clustering accuracy on dimensionality reduced datasets.

Now imagine if our dataset has hundreds of thousands or millions of dimensions, this is really where using dimensionality reduction would shine. (Big Data)

#K_Means on the Original Dataset
To make this point clearer, lets  perform clustering on the original dataset and measure how varying the number of dimensions we pass into the clustering algorithm affects clustering accuracy.

In [ ]:
#k-means  - Accuracy as the number of components varies
#On the "original (non dimensioanlity reduced dataset)

n_clusters = 20
n_init= 10
max_iter= 300
tol= 0.0001
random_state= 2018
n_jobs= 2

kmeans_inertia= pd.DataFrame(data= [], index= [9,49,99,199,299,399,
                                               499,599,699,784], columns = ['inertia'])
overallaccuracy_kmeansDF= pd.DataFrame(data= [], index= [9, 49, 99, 199, 299, 399,
                                                         499, 599, 699, 784], columns= ['overall_accuracy'])

for cutoffnumber in [9, 49, 99, 199, 299, 399, 499, 599, 699, 784]:
  kmeans= KMeans(n_clusters= n_clusters, n_init= n_init, max_iter= max_iter, 
                 tol= tol, random_state= random_state, n_jobs= n_jobs)
  cutoff = cutoffnumber
  kmeans.fit(x_train.loc[:, 0: cutoff])
  kmeans_inertia.loc[cutoff]= kmeans.inertia_

x_train_kmeansclustered= kmeans.predict(x_train.loc[:, 0:cutoff])
x_train_kmeansclustered= pd.DataFrame(data= x_train_kmeansclustered, index = x_train.index, columns= ['cluster'])

countbycluster_kmeans, countbylabel_kmeans, countmostfreq_kmeans, accuracydf_kmeans, overallaccuracy_kmeans, accuracybylabel_kmeans= analyzecluster(
    x_train_kmeansclustered, y_train)

overallaccuracy_kmeansDF.loc[cutoff]= overallaccuracy_kmeans

This is going to plot the clustering accuracy at the different original dimensions.

As the plot shows, the clustering accuracy is very poor at lower dimensions BUT improves to nearly 70% only only as the number of dimensions climbs to six hundred dimensions.

*In the PCA case, clustering accuracy was roughly 70% even at 10 dimensions, demonstrating the power of dimensionality reduction to densely capture salient informationn in the original dataset.

#Hierarchical Clustering
This approach does not require us to pre-commit to a particular number of clusters. <br>
Instead, we can choose how many clusters we would like after hierarchical clustering has finished running.

This algorithm will build what's called a dendrogram, which can be depicted as an upside down tree where the leaves are at the bottom and the branch is at the tree trunk is at the top.

The leaves at the very are individual instances in the dataset. This algorithm then joins the leaves together = as we move vertically up the upside down tree- based on how similar they are to each other.<br>
*The instances (or group of instances) that are most similar to each other are joined sooner, while the instances that are not as similar are joined later.

With this iterative process, all of the instances are eventually linked together forming the "single trunk fof the tree".


**This vertical depiction is very helpful seeing thatg we can view the dendrogram and determine where we want to cut the tree. The lower we, the more individual branches we are left with (more clusters).<br>
If we want fewer clusters, we can cut higher on the dendrogram, closer to the single trunk at the very top of this upside down.
<br>
This is similar to choosing the number of k-clusters in the k-means clustering algorithm.

**WE WILL EXPLORE A VERSION OF THIS ALGORITHM CALLED AGGLOMERATIVE HIERARCHICAL CLUSTERING ** (known as FastCluster when coding)

In [ ]:
import fastcluster
from scipy.cluster.hierarchy import dendrogram, cophenet
from scipy.spatial.distance import pdist

cutoff= 100  #only using 100 components
z= fastcluster.linkage_vector(x_train_pca.loc[:, 0: cutoff], method= 'ward', metric= 'euclidean')
z_dataframe= pd.DataFrame(data= z, columns= ['cluster_one', 'cluster_two', 'distance', 'New_Cluster_Size'])

The algorithm will return a maxtrix Z.<br>
The algorithm treats each observation in our 50,000 mnist dataset as a single point cluster, and in each iteration of training, the algorithm will merge the two clusters that have the smallest distance between them.

*Initially, the algorithm is just merging single point clusters together, but as it proceeds, it will merge multipoint clusters with either single point or multi-point clusters. <br>
Eventually through this iterative process, the all the clusters are merge together, forming the trunk in the upside down tree. (dendrogram)

#Evaluating the Clustering Results
Now that we have the dendrogram in place, lets determine where to cut off of the dendrogram to make the number of clusters we desire.

Now in order to compare this type of algorithm to the k-means clustering algorithm, lets cut the dendrogram to have exactly 20 clusters.<br>
We will then use the clustering accuracy metric- defined in the k-means section- to judge how homogenous the hierarchical clustering clusters are.

To create the clusters we desire the dendrogram, lets pull in the fcluster lib from scipy.<br>
We need to specify the distance threshold of the dendrogram to determine how many "distinct" clusters we are left with.<br>
*The larger  the distance threshold, the fewer clusters we will have.

To get exactly 20 clusters, we need to experiment with the distance threshold.<br>
*The fcluster lib will take our dendrogram and cut it with the distance threshold we specify. Each observation in the 50,000 observations MNIST digits dataset will get a clsuter label, adn we will store these in a pandas dataframe

In [ ]:
from scipy.cluster.hierarchy import fcluster

distance_threshold= 160
clusters= fcluster(z, distance_threshold, criterion= 'distance')
x_train_hierclustered= pd.DataFrame(data= clusters, index= x_train_pca.index,
                                    columns= ['cluster'])

#lets verify that there are exaclty 20 distinct clusters, given our choice of distance threshold
print('Number of distinct clusters: ', len(x_train_hierclustered['cluster'].unique()))

As expected, this confirms 20 clusters.<br>
Now lets evaluate the results

In [ ]:
countbycluster_hierclust, countbylabel_hierclust, countmostfreq_hierclust, accuracydf_hierclust, overallallaccuracy_hierclust, accuracylabel_hierclust= analyze_cluster(
    x_train_hierclustered, y_train)
print(('Overall accuracy from hierarchical clustering: ', overallaccuracy_hierclust)

As we can see, the overall accuracy is approx. 77%...even better than the approx for k-means algorithm.

Lets also assess the accuracy rate by cluster. <br> 
As we can see, some clusters have an accuracy score as high as 100& while other as low as 50%. This gives us an overall accuracy score of roughly 77%...not bad at all!

Overall, the hierarchical clustering algorithm performs well on MNIST dataset. And this was all done without using ANY labels.

This is how it would work on real-world examples:
  - Step 1: Apply dimensionality reduction first (such as PCA)
  - Step 2: perform clustering (such as hierarchical clustering)
  - Step 3: Hand label a few points per cluster

*So long as the clusters we homogenous enough (same kind or alike), the few hand labels we generated could be applied automatically to all of the other images in the cluster.

All of a sudden, without much effort, we could have labeled all of the images in our 50,000 dataset with a near 77% accuracy. <br>

*#THIS IS IMPRESSIVE AND HIGHLIGHTS THE POWER OF UNSUPERVISED LEARNING! **bold text**

#DBSCAN
This final clustering algorithm groups based on the density of points.<br>
In k-meand and herarchical clustering algorithms, all points had to be clustered in some way shape or form...***and outliers we poorly dealt with.***<br>

**In DBSCAN, we can explicitly label points as outliers and avoid having tok cluster them in the first place....THIS IS VERY POWERFUL!!**

Compared to other clustering algorithms, DBSCAN is much less prone to the distortion typically caused by outliers in the data.<br>
(Also, like hierarchical clustering, and unlike k-means, we do not need to prespecify the number of clusters)

All points that do not get grouped- either as the core border points of a cluster- are labeled as outliers.

In this example, we are going to apply DBSCAN to the first 100 compenents within our dataset.

In [ ]:
#DBSCAN
from sklearn.cluster import DBSCAN

eps= 3
min_samples = 5     #default value
leaf_size= 30
n_jobs = 4


db= DBSCAN(eps = eps, min_samples = min_samples, leaf_size= leaf_size,
           n_jobs= n_jobs)
cutoff= 99     #100 components we are using (counting 0 as the number 1)
x_train_pca_dbscan_clustered= db.fit_predict(x_train_pca.loc[:,0:cutoff])
x_train_pca_dbscan_clustered= pd.DataFrame(data= x_train_pca_dbscan_clustered, index= x_train.index,
                                           columns= ['cluster'])

countbycluster_dbscan, countbylabel_dbscan, countmostfreq_dbscan, accuracyDF_dbscan, overallaccuracy_dbscan, accuracybylabel_dbscan= analze_cluster(
    x_train_pca_dbscan, y_train)

print('Overall Accuracy DBSCAN: ' overallaccuracy_dbscan)

As we can see, the accuracy is very poor compared to k-means and hierarchical clustering.<br>
We can fidget around with some of the parameters such as "eps" and "min_samples" to improve results, but it appears thay DBSCAN is poorly suited to cluster the observations for this particular dataset.

#Conclusion
In this notebook, we introduced three major clustering algorithms known as:
  - k-means clustering
  - hierachical clustering
  - DBSCAN

The first two algorithms performed the best, with hierachical clustering coming out on top.